In [ ]:
# Import necessary libraries
import h2o
from h2o.automl import H2OAutoML

# Load the data
#train = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')
#test = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv')

# Initialize the H2O cluster
h2o.init()

# Load your data into a H2OFrame
data = h2o.import_file('/kaggle/input/optiver-trading-at-the-close/train.csv')

# Specify the response and predictor columns
response_column = 'target'
predictor_columns = data.columns.remove(response_column)

# Split the data into training and validation sets
train, val = data.split_frame(ratios=[0.8], seed=1)

# Initialize and train the H2OAutoML model
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=predictor_columns, y=response_column, training_frame=train)

# View the AutoML Leaderboard
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))

# Predict on the validation set
preds = aml.predict(val)

# Convert the predictions back to a pandas dataframe (optional)
preds_df = h2o.as_list(preds)



Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu120.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu120.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpguw83q0o
  JVM stdout: /tmp/tmpguw83q0o/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpguw83q0o/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,1 month and 13 days
H2O_cluster_name:,H2O_from_python_unknownUser_qnstf7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
11:06:00.619: _train param, Dropping bad and constant columns: [row_id]


11:06:02.759: XGBoost_1_AutoML_1_20231006_110600 [XGBoost def_2] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_1_AutoML_1_20231006_110600_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.

11:06:02.776: _train param, Dropping bad and constant columns: [row_id]

██████████████████████████████████████████████████████████
11:10:23.516: _train param, Dropping bad and constant columns: [row_id]

████